In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
from typing import List
from typing import Optional
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
mapping_file = "../../../data_points/Naarni VehicleID_RegNo_links - Vehicle_mapping.csv"
try:
    df_mapping = pd.read_csv(mapping_file)
except FileNotFoundError:
    print(f"Error: Mapping file '{mapping_file}' not found. Cannot enrich data.")
    # Create an empty mapping table to allow the rest of the script to run without crashing
    df_mapping = pd.DataFrame(columns=["id", "reg_num", "customer", "model"])
else:
    df_mapping = df_mapping.rename(columns={
        "Device No.": "id",
        "Registration No": "reg_num",
        "Customer": "customer",
        "Model": "model"
    })
    # Ensure the merge key ('id') is a string to match the chunks
    if "id" in df_mapping.columns:
        df_mapping["id"] = df_mapping["id"].astype(str)
        df_mapping = df_mapping[["id", "reg_num", "customer", "model"]]
    else:
        print("Warning: 'Device No.' column not found in mapping file.")
        df_mapping = pd.DataFrame(columns=["id", "reg_num", "customer", "model"])

print(f"Loaded mapping table with {len(df_mapping)} entries.")
# df_mapping is now ready to be passed into the processing function

df_mapping.head()

Loaded mapping table with 27 entries.


,id,reg_num,customer,model
0,11,AP39WF8593,FB Guntur,12.5
1,9,AP39WG0252,FB Guntur,12.5
2,7,AP39WF8589,FB Guntur,12.5
3,13,AP39WF8584,FB Guntur,12.5
4,14,AP39WG0271,FB Guntur,12.5


In [3]:
CORE_COLS = [
    "id", "timestamp", "dt",
    "vehiclereadycondition", "gun_connection_status", "ignitionstatus",
    "vehicle_speed_vcu", "gear_position",
    "bat_soc", "soh", "total_battery_current",
    "pack1_cellmax_temperature", "pack1_cell_min_temperature",
    "pack1_maxtemperature_cell_number", "pack1_celltemperature_cellnumber",
    "bat_voltage", "cellmax_voltagecellnumber", "cellminvoltagecellnumber", 
    "cell_min_voltage","cell_max_voltage",
]

In [4]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [5]:
def rename_battery_temp_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Uses df.rename(inplace=False), creating one copy, which is fine for chunks
    rename_map = {
        "pack1_cellmax_temperature": "batt_maxtemp",
        "pack1_cell_min_temperature": "batt_mintemp",
        "pack1_maxtemperature_cell_number":"batt_maxtemp_tc", 
        "pack1_celltemperature_cellnumber":"batt_mintemp_tc",
        "cell_max_voltage":"batt_maxvolt",
        "cellmax_voltagecellnumber":"batt_maxvolt_cell",
        "cell_min_voltage":"batt_minvolt",
        "cellminvoltagecellnumber":"batt_minvolt_cell", 
    }
    existing = {k: v for k, v in rename_map.items() if k in df.columns}
    if not existing:
        return df
    return df.rename(columns=existing)

In [6]:
def impute_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    # This uses df.loc, which *can* cause copies but is the necessary imputation logic
    df = df.sort_values(["id", "timestamp"])
    for vid, grp in df.groupby("id", sort=False):
        idx = grp.index
        # Imputation logic...
        for col, limit in [("batt_maxtemp", 60), ("batt_mintemp", 60), 
                           ("batt_maxtemp_tc",60),("batt_mintemp_tc",60),
                           ("batt_maxvolt", 30), ("batt_minvolt", 30), 
                           ("batt_maxvolt_cell",30),("batt_minvolt_cell",30),
                           ("bat_voltage", 20), ("bat_soc", 300), ("soh", 300)]:
            if col in df.columns:
                df.loc[idx, col] = grp[col].ffill(limit=limit)
        
        if "total_battery_current" in df.columns:
            df.loc[idx, "total_battery_current"] = grp["total_battery_current"].interpolate(
                limit=10, limit_direction="both"
            )
        for col in ["vehiclereadycondition", "gun_connection_status"]:
            if col in df.columns:
                 df.loc[idx, col] = grp[col].ffill().bfill()

    return df # Returns the modified chunk

In [7]:
def prepare_df_with_state(df: pd.DataFrame, df_mapping:pd.DataFrame) -> pd.DataFrame:
    # CRITICAL: This line creates the necessary copy, but now it's only a small chunk!
    out = df.copy() 
    
    # 1. Merge Vehicle Information
    out["id"] = out["id"].astype(str) # Ensure merge key type consistency
    out = out.merge(
        df_mapping, 
        on="id", 
        how="left", 
        validate="m:1"
    )

    # --- NEW: Fill Missing Mapping Information ---
    # Create the standby values based on the 'id' column for rows where the merge failed (NaNs)
    
    # 1a. Fill 'reg_num'
    # Use .mask to replace NaN only where 'reg_num' is NaN, using the formatted ID
    out["reg_num"] = out["reg_num"].mask(
        out["reg_num"].isna(), 
        "REGNUM_" + out["id"].astype(str)
    )
    
    # 1b. Fill 'customer'
    out["customer"] = out["customer"].mask(
        out["customer"].isna(), 
        "CUST_" + out["id"].astype(str)
    )

    # 1c. Fill 'model'
    out["model"] = out["model"].mask(
        out["model"].isna(), 
        "MDL_" + out["id"].astype(str)
    )    
    
    # --- CRITICAL FIX: Ensure new columns are explicitly string type for PyArrow ---
    out["reg_num"] = out["reg_num"].astype(str)
    out["customer"] = out["customer"].astype(str)
    out["model"] = out["model"].astype(str)
    # -----------------------------------------------------------------------------

    # 2. Basic Cleanup & Sorting
    # --- NEW: TIMEZONE CONVERSION TO IST ---
    # Assuming the original timestamps are stored as naive UTC time
    out["timestamp"] = (
        out["timestamp"]
        .dt.tz_localize("UTC")
        .dt.tz_convert("Asia/Kolkata")
    )    
    out["timestamp"] = pd.to_datetime(out["timestamp"], errors="coerce")
    out = out.dropna(subset=["timestamp"]).sort_values(["id", "timestamp"]).reset_index(drop=True)
    
    # 3. Mode calculation
    gcs_raw = out["gun_connection_status"]
    gcs_num = pd.to_numeric(gcs_raw, errors="coerce")
    gcs_str = gcs_raw.astype(str).str.strip().str.lower()
    gun_connected = (gcs_num == 1) | gcs_str.isin({"1", "true", "yes", "y", "connected", "on"})
    out["mode"] = np.where(gun_connected, "CHARGING", "DISCHARGING")
    
    # 4. Delta calculations
    for col in ["batt_maxtemp", "batt_mintemp", "batt_maxvolt", "batt_minvolt"]:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors="coerce")
            
    out["batt_temp_delta"] = out["batt_maxtemp"] - out["batt_mintemp"]
    out["volt_delta_mv"] = round((out["batt_maxvolt"] - out["batt_minvolt"]) * 1000.0,0)
    out["dt_sec"] = out.groupby("id")["timestamp"].diff().dt.total_seconds().fillna(0)
    
    # 5. --- NEW: Create Feature Buckets (Binning) ---
    temp_bins = [-np.inf, 28, 32, 35, 40, np.inf]
    temp_labels = ["<28", "28–32", "32–35", "35–40", ">40"]

    delta_bins = [-np.inf, 2, 5, 8, np.inf]
    delta_labels = ["<2", "2–5", "5–8", ">8"]

    volt_bins = [0, 10, 20, 30, np.inf]
    volt_labels = ["0–10", "10–20", "20–30", ">30"]
    
    # **NOTE:** Corrected df to use the local working DataFrame 'out'
    out["maxtemp_bucket"] = pd.cut(out["batt_maxtemp"], bins=temp_bins, labels=temp_labels)
    out["temp_delta_bucket"] = pd.cut(out["batt_temp_delta"], bins=delta_bins, labels=delta_labels)
    out["volt_delta_bucket"] = pd.cut(out["volt_delta_mv"], bins=volt_bins, labels=volt_labels)
    # -------------------------------------------------

    cols_keep = [
        "id", "reg_num","customer","model","timestamp","dt_sec","mode",
        "vehiclereadycondition", "gun_connection_status",
        "batt_maxtemp", "batt_mintemp", "batt_temp_delta","maxtemp_bucket","temp_delta_bucket",
        "batt_maxvolt", "batt_minvolt", "volt_delta_mv","volt_delta_bucket",
        "batt_maxtemp_tc","batt_mintemp_tc",
        "batt_maxvolt_cell","batt_minvolt_cell",
        "bat_voltage", "total_battery_current",
        "bat_soc", "soh",
    ]
    cols_keep = [c for c in cols_keep if c in out.columns]
    # This final filter also creates a copy, but again, only of the small chunk
    out = out[cols_keep]
    return out

In [8]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(output_dir, utc_start, utc_end, core_cols):
    """Sets up DuckDB connection and SQL query."""
    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))
    # Only select the columns you need for Stage 1 processing
    column_list = ", ".join([f'"{c}"' for c in core_cols])
    
    # CRITICAL: Predicate Pushdown filter on the internal 'timestamp' column
    sql_query = f"""
        SELECT {column_list}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND 
            "timestamp" < '{utc_end.isoformat()}'
    """
    return duckdb.connect(), sql_query

In [9]:
def run_stage1_data_setup(analysis_start_date_str: str, 
                          analysis_end_date_str: str, 
                          zip_path: Path, 
                          extraction_dir: Path,
                          force_extraction: bool = False) -> tuple[datetime, datetime, int]:
    """
    Handles date range setup, IST-to-UTC conversion, file extraction, 
    and checks if data is available for processing.
    
    Args:
        analysis_start_date_str: Start date in YYYY-MM-DD format.
        analysis_end_date_str: End date in YYYY-MM-DD format.
        zip_path: Path to the source ZIP file.
        extraction_dir: Target directory for extracted Parquet files.
        force_extraction: If True, always clean and re-extract files. 
                          If False, skips extraction if the directory exists.
    
    Returns: (utc_start, utc_end, file_count)
    """
    
    # 1. Date Parsing and UTC Conversion (Assuming +5:30 IST offset)
    target_date = datetime.strptime(analysis_start_date_str, "%Y-%m-%d").date()
    ist_start = datetime.combine(target_date, datetime.min.time())
    
    end_date_obj = datetime.strptime(analysis_end_date_str, "%Y-%m-%d").date()
    ist_end = datetime.combine(end_date_obj, datetime.min.time()) + timedelta(days=1)
    
    utc_start = ist_start - timedelta(hours=5, minutes=30)
    utc_end = ist_end - timedelta(hours=5, minutes=30)
    
    logging.info(f"🔍 Analysis window (UTC): {utc_start} → {utc_end}")

    # 2. FILE EXTRACTION CONTROL
    file_count = 0
    
    if extraction_dir.exists() and not force_extraction:
        logging.info("♻️ Skipping file extraction: Directory exists and force_extraction=False.")
        # Recursively count all .parquet files in the existing directory
        file_count = len(list(extraction_dir.rglob('*.parquet')))
        if file_count > 0:
             logging.info(f"✅ Found {file_count} existing files. Proceeding to DuckDB loading.")
        
    else:
        # If directory doesn't exist, or force_extraction is True, run the full extraction.
        logging.info("🔄 Running full extraction (Cleanup + Extract)...")
        # This relies on the robust `extract_files_to_disk` function
        file_count = extract_files_to_disk(zip_path, extraction_dir)
        
    # 3. Validation Check
    if file_count == 0:
        logging.warning("🛑 Skipping analysis: No files were found.")
        sys.exit() # Exit the script cleanly if no files were found

    return utc_start, utc_end, file_count

In [10]:
# --- CONFIGURATION (Ensure these are defined at the top of your script) ---
ZIP_FILE_PATH = "../../../data_points/naarni75_cpoall.zip" 
EXTRACTION_DIR = Path("../extracted_parts") 
# --------------------------------------------------------------------------

# --- Set 75-Day Date Range (Using your target dates) ---
analysis_start_date_str = "2025-09-01" 
# NOTE: Using 2025-11-14 since 75 days starts on 2025-09-01 and ends on 2025-11-14.
# Using 2025-11-15 will include the start of the 76th day (if data exists).
analysis_end_date_str = "2025-11-15"   
# --------------------------------------------------------

# NEW STAGE 1 EXECUTION:
utc_start, utc_end, file_count = run_stage1_data_setup(
    analysis_start_date_str=analysis_start_date_str,
    analysis_end_date_str=analysis_end_date_str,
    zip_path=ZIP_FILE_PATH,
    extraction_dir=EXTRACTION_DIR,
    force_extraction = False)

2025-11-23 02:15:31 - INFO - 🔍 Analysis window (UTC): 2025-08-31 18:30:00 → 2025-11-15 18:30:00
2025-11-23 02:15:31 - INFO - ♻️ Skipping file extraction: Directory exists and force_extraction=False.
2025-11-23 02:15:31 - INFO - ✅ Found 474 existing files. Proceeding to DuckDB loading.


In [11]:
# 2. SETUP DUCKDB QUERY
conn, sql_query = setup_duckdb_query(EXTRACTION_DIR, utc_start, utc_end, CORE_COLS)

# A quick DuckDB query to get the distinct IDs from the 75-day filtered dataset
get_ids_query = f"""
    SELECT DISTINCT id 
    FROM ({sql_query})
"""
# Fetch the list of IDs (this is a very small amount of data)
vehicle_ids = conn.execute(get_ids_query).fetchdf()["id"].astype(str).tolist()
# vehicle_ids = ['3','16','18','19','32','42','6','7','9','11','12','13','14','15','20','25','27','28','29','30','31','33','35','41','46']

logging.info(f"✅ Found {len(vehicle_ids)} unique vehicle IDs in the dataset.")
print(sorted(vehicle_ids))

2025-11-23 02:15:31 - INFO - ✅ Found 29 unique vehicle IDs in the dataset.


['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '41', '42', '46', '6', '7', '9']


In [12]:

# NOTE: The functions 'duckdb_chunk_generator', 'rename_battery_temp_columns', 
# 'impute_missing_values', 'prepare_df_with_state', and 'free_mem' 
# MUST be defined in your Jupyter Notebook environment before calling this.

def process_and_save_data(
    conn: duckdb.DuckDBPyConnection, 
    sql_query: str, 
    chunk_size: int, 
    parquet_feather_path: str,
    vehicle_ids: List[str],
    df_mapping: pd.DataFrame,
    skip_if_exists: bool = True
) -> int:
    """
    Executes the memory-safe chunked processing loop for Stage 1.
    
    Loads raw data from DuckDB query (75 days) in chunks, preprocesses (renaming, 
    imputation, state preparation), filters by vehicle ID, and saves the result 
    to a single Feather file on disk.
    
    Args:
        conn: The active DuckDB connection.
        sql_query: The SQL query containing the timestamp filter.
        chunk_size: The maximum number of rows to load into RAM per chunk.
        feather_path: The name/path of the final output Feather file.
        vehicle_ids: List of specific vehicle IDs to include.
        
    Returns:
        The total number of rows processed and saved to the Feather file.
    """
    
    logging.info(f"🧠 Processing data in {chunk_size:,} row chunks...")
    logging.info(f"💾 Output path set to: {parquet_feather_path}")
    
    output_path = Path(parquet_feather_path)    
    if skip_if_exists and output_path.exists():
            conn.close() # Close the connection before returning
            
            # Safely retrieve the row count from the existing file using DuckDB
            conn_count = duckdb.connect()
            try:
                total_rows = conn_count.execute(f"SELECT count(*) FROM '{parquet_feather_path}'").fetchone()[0]
                logging.info(f"✅ Skipping processing: Found existing file with {total_rows:,} rows.")
                return total_rows
            except Exception as e:
                logging.error(f"Error reading row count from existing file: {e}. Re-running processing.")
                # If the count fails, let the function continue and overwrite the file
            finally:
                conn_count.close()

    # 1. Initialization
    first_chunk = True
    total_processed_rows = 0

    # 2. Use the generator to stream data
    for chunk in duckdb_chunk_generator(conn, sql_query, chunk_size):
        
        df_chunk = chunk
        
        # 3.1 Renaming and ID cleanup
        df_chunk = rename_battery_temp_columns(df_chunk)
        
        if "id" in df_chunk.columns:
            df_chunk["id"] = df_chunk["id"].astype(str)
            
            # Filter by vehicle ID on the small chunk
            if vehicle_ids: # Only filter if the list is NOT empty
                df_chunk = df_chunk[df_chunk["id"].isin(vehicle_ids)]
            
            # --- Type Casting Optimization: Explicitly cast types after ID filter ---
            df_chunk = df_chunk.convert_dtypes() # Use Pandas native types
        
        # 3.2 Imputation and State Preparation
        df_chunk["timestamp"] = pd.to_datetime(df_chunk["timestamp"], errors="coerce")
        df_chunk = impute_missing_values(df_chunk)
        
        # State preparation uses df.copy(), which is now safe on the small chunk
        df_chunk_with_state = prepare_df_with_state(df_chunk,df_mapping)
        
        # 3.3 Memory Check and Save
        if df_chunk_with_state.empty:
            del df_chunk, df_chunk_with_state
            gc.collect()
            free_mem()
            continue
            
        # 3.4 Save Chunk (Append to Feather file)
        total_processed_rows += len(df_chunk_with_state)
        
        if first_chunk:
            # Overwrite file if it's the first chunk
            # df_chunk_with_state.to_feather(feather_path, compression='zstd')
            df_chunk_with_state.to_parquet(parquet_feather_path, compression='zstd', index=False)
            first_chunk = False
        else:
            # Append all subsequent chunks (Requires pyarrow 'a' mode)
            # df_chunk_with_state.to_feather(feather_path, compression='zstd', mode='a')
            # df_chunk_with_state.to_parquet(parquet_feather_path, compression='zstd', index=False, engine='fastparquet', mode='append')
            fastparquet.write(
                    parquet_feather_path, 
                    df_chunk_with_state, 
                    compression='zstd', 
                    write_index=False, # Equivalent to index=False in pandas
                    append=True        # The correct argument for appending with fastparquet
                )            
            
        logging.info(f"   Processed {total_processed_rows:,} rows so far...")
        
        # 3.5 Clean up memory after processing each chunk (CRITICAL STEP)
        del df_chunk, df_chunk_with_state
        gc.collect()
        free_mem() # Aggressively free memory
        
    conn.close()
    
    return total_processed_rows

In [ ]:
# --- ASSUMING run_stage1_data_setup WAS CALLED AND RETURNED utc_start, utc_end ---
# Example configuration that needs to be available:
# EXTRACTION_DIR = Path("../extracted_parts") 
# CORE_COLS = [...]
# CRITICAL FIX: Drastically reduced chunk size to prevent memory spike
CHUNK_SIZE = 10_000 # Process 50,000 rows max at any time


# 1. Setup DuckDB Query (as shown previously)
conn, sql_query = setup_duckdb_query(EXTRACTION_DIR, utc_start, utc_end, CORE_COLS)

# 2. Define Inputs
# output_feather_file = "df_with_state_30days.feather"
output_parquet_file = "df_with_state.parquet"

# 3. Run the memory-safe processing loop
total_rows = process_and_save_data(
    conn=conn,
    sql_query=sql_query,
    chunk_size=CHUNK_SIZE,
    parquet_feather_path=output_parquet_file,
    vehicle_ids=vehicle_ids,
    df_mapping=df_mapping,
    skip_if_exists =True
)

logging.info(f"🎉 Final DataFrame saved. Total rows processed: {total_rows:,}")
logging.info("✅ Data Processing (Stage 1) complete. Feather file ready for analysis.")

2025-11-23 02:15:31 - INFO - 🧠 Processing data in 10,000 row chunks...
2025-11-23 02:15:31 - INFO - 💾 Output path set to: df_with_state_30days.parquet
2025-11-23 02:15:31 - INFO - ✅ Skipping processing: Found existing file with 52,888,998 rows.
2025-11-23 02:15:31 - INFO - 🎉 Final DataFrame saved. Total rows processed: 52,888,998
2025-11-23 02:15:31 - INFO - ✅ Data Processing (Stage 1) complete. Feather file ready for analysis.


In [14]:
def read_parquet_subset(parquet_path: str, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
    """
    Reads a subset of the processed Feather file using a date filter
    applied directly by DuckDB (predicate pushdown).
    
    Args:
        feather_path: Path to the processed Feather file.
        start_dt: Start datetime for the filter (inclusive).
        end_dt: End datetime for the filter (exclusive).
        
    Returns:
        A new DataFrame containing only the filtered data.
    """
    logging.info(f"Loading data subset from {start_dt} to {end_dt}...")
    
    # Use DuckDB to query the Feather file directly on disk
    con = duckdb.connect()
    
    # The SQL query filters rows on the disk file based on the 'timestamp' column.
    sql_query = f"""
        SELECT *
        FROM read_parquet('{parquet_path}')
        WHERE 
            "timestamp" >= '{start_dt.isoformat()}' AND 
            "timestamp" < '{end_dt.isoformat()}'
    """
    
    # Fetch the filtered, smaller DataFrame
    df_subset = con.execute(sql_query).fetchdf()
    con.close()
    
    logging.info(f"✅ Loaded {len(df_subset):,} rows for the requested subset.")
    return df_subset

In [15]:
# # Define the 30-day filter window
# filter_start_date = datetime(2025, 9, 1)
# filter_end_date = datetime(2025, 9, 2) # Exclusive end date

# # 1. Load the filtered subset safely
# df_subset = read_parquet_subset(
#     parquet_path="df_with_state_30days.parquet",
#     start_dt=filter_start_date,
#     end_dt=filter_end_date
# )

# display(df_subset.head())
# 2. Perform analysis on the smaller DataFrame
# ... run get_veh_temp_hotspots, compute_batt_condition_metrics, etc., on df_subset ...

# # 3. Aggressive memory cleanup after use (CRITICAL)
# del df_subset
# gc.collect()
# free_mem()

In [16]:
# df_subset.timestamp.min(),df_subset.timestamp.max()

In [17]:
# --- FIXED CATEGORY DEFINITIONS ---
# These fixed labels ensure your output columns are consistent across all data chunks.
FIXED_CATEGORIES = {
    'maxtemp_bucket': ["<28", "28–32", "32–35", "35–40", ">40"],
    'temp_delta_bucket': ["<2", "2–5", "5–8", ">8"],
    # 💥 UPDATED BUCKET: Using the new labels for volt_delta_bucket 💥
    'volt_delta_bucket': ["0–10", "10–20", "20–30", ">30"] 
}
# ----------------------------------

def create_session_report(
    df: pd.DataFrame, 
    timezone: str = 'Asia/Kolkata',
    min_discharging_duration_min: float = 1.0
) -> pd.DataFrame:
    """
    Performs sessionization and calculates the percentage distribution of 
    key operational buckets per session using a fixed set of categories.
    """
    # 1. CRITICAL STEP: Ensure data is sorted by ID and Timestamp
    df = df.sort_values(['id', 'timestamp']).reset_index(drop=True)
    
    # Ensure 'timestamp' is timezone-aware IST
    if not isinstance(df['timestamp'].dtype, pd.DatetimeTZDtype):
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True).dt.tz_convert(timezone)
        except Exception:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # 2. Identify the start of a new session
    df['id_change'] = df['id'].ne(df['id'].shift())
    df['mode_change'] = df['mode'].ne(df['mode'].shift())
    df['session_start'] = df['id_change'] | df['mode_change']
    df['unique_session_id'] = df['session_start'].cumsum()

    # 3. Aggregate Session Metrics (Non-bucket columns)
    session_df = df.groupby('unique_session_id').agg(
        id=('id', 'first'),
        reg_num=('reg_num', 'first'),
        customer=('customer', 'first'),
        model=('model', 'first'),
        mode=('mode', 'first'),
        start_time=('timestamp', 'min'),
        end_time=('timestamp', 'max'),
    ) # Unique_session_id is the index

    # 4. --- NEW: Calculate Percentage of Buckets per Session using FIXED CATEGORIES ---
    bucket_cols = ['maxtemp_bucket', 'temp_delta_bucket', 'volt_delta_bucket']
    agg_dfs = []
    
    for col in bucket_cols:
        if col in df.columns and col in FIXED_CATEGORIES:
            
            # --- Key Fix: Convert to CategoricalDtype with fixed categories ---
            # This ensures all categories are present, even those with zero count.
            df[col] = pd.Categorical(df[col], categories=FIXED_CATEGORIES[col], ordered=True)
            # -------------------------------------------------------------------
            
            # Calculate value counts normalized to get percentages
            pct_series = round(df.groupby('unique_session_id')[col].value_counts(normalize=True) * 100,2)
            
            # Unstack the bucket labels to create new columns, fill missing (zero) categories
            pivot_df = pct_series.unstack(fill_value=0)
            
            # Rename columns for clear identification and safe use (e.g., replaces '–' with '_')
            pivot_df.columns = [
                f"{col}_{str(c).replace('–', '_').replace('<', 'lt').replace('>', 'gt')}_pct" 
                for c in pivot_df.columns
            ]
            agg_dfs.append(pivot_df)

    # Merge all percentage tables back into the main session_df
    if agg_dfs:
        for agg_df in agg_dfs:
            session_df = session_df.join(agg_df, how='left')
    
    session_df = session_df.reset_index(drop=True) # Now reset index

    # 5. Final Calculations and Formatting
    
    # Calculate Duration in MINUTES (seconds / 60) and round to 2 decimal places
    session_df['duration'] = (
        (session_df['end_time'] - session_df['start_time']).dt.total_seconds() / 60
    ).round(2)

    # Add sequential session number (e.g., 1, 2, 3...) per vehicle ID
    session_df['session'] = session_df.groupby('id').cumcount() + 1
    
    # 6. Final Output Selection and Formatting
    
    session_report = session_df.copy()

    # Define the core columns to ensure they are first
    core_cols = ['id', 'reg_num', 'customer', 'model', 'mode', 'session', 'start_time', 'end_time', 'duration']
    
    # Order columns: core columns first, then new percentage columns
    new_cols = [c for c in session_report.columns if c not in core_cols]
    session_report = session_report[core_cols + new_cols]
    
    # --- FINAL FORMATTING ---
    # Round times to seconds precision (floor)
    session_report['start_time'] = session_report['start_time'].dt.floor('S')
    session_report['end_time'] = session_report['end_time'].dt.floor('S')

    # Drop Timezone information (+05:30)
    session_report['start_time'] = session_report['start_time'].dt.tz_localize(None)
    session_report['end_time'] = session_report['end_time'].dt.tz_localize(None)
    # -------------------------

    return session_report

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import gc
import logging # Required for managing log levels
from tqdm.auto import tqdm 
import time # For timing the execution

# NOTE: The create_session_report and read_parquet_subset functions 
# must be defined/imported before running this code.

def run_multi_day_session_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
) -> pd.DataFrame:
    """
    Iteratively loads, processes, and aggregates session reports for a defined number of days,
    displaying a progress bar and suppressing INFO logging during the process.
    """
    all_reports = []
    total_rows_ingested = 0 # <--- ADD THIS LINE HERE

    # --- Logging Suppression: Temporarily set log level to WARNING to hide INFO messages ---
    current_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING) 
    
    try:
        progress_bar = tqdm(range(num_days), desc="Processing Daily Sessions", unit="day")
        
        for i in progress_bar:
            current_start_dt = start_date + timedelta(days=i)
            current_end_dt = current_start_dt + timedelta(days=1)
            
            progress_bar.set_postfix_str(f"Date: {current_start_dt.strftime('%Y-%m-%d')}")
            
            # 1. Load the small subset (Logging is suppressed here)
            df_subset = read_parquet_subset(
                parquet_path=parquet_path,
                start_dt=current_start_dt,
                end_dt=current_end_dt
            )
            
            if df_subset.empty:
                del df_subset
                gc.collect()
                continue
            
            total_rows_ingested += len(df_subset)   #count rows being processed

            # 2. Process the subset
            session_report_chunk = create_session_report(df_subset)
            
            # 3. Aggregate the result
            all_reports.append(session_report_chunk)
            
            # 4. Explicit Memory Management
            del df_subset 
            gc.collect()  
            
    finally:
        # --- Restore Logging Level ---
        logging.getLogger().setLevel(current_log_level)
        
    # 5. Final Concatenation
    if all_reports:
        final_report_df = pd.concat(all_reports, ignore_index=True)
        return final_report_df, total_rows_ingested
    else:
        return pd.DataFrame(), total_rows_ingested

# =========================================================================
# --- Execution Example ---
# =========================================================================

# Define the 75-day process window
filter_start_date = datetime(2025, 9, 1) # Start date
total_days_to_process = 75

print(f"--- Starting 75-Day Session Analysis ---")
print(f"Processing range: {filter_start_date.strftime('%Y-%m-%d')} to {(filter_start_date + timedelta(days=total_days_to_process-1)).strftime('%Y-%m-%d')}")
print("-" * 40)

start_time = time.time()

# Run the iterative analysis
session_report_df, total_rows = run_multi_day_session_analysis(
    parquet_path="df_with_state.parquet",
    start_date=filter_start_date,
    num_days=total_days_to_process
)

end_time = time.time()
elapsed_time_sec = end_time - start_time
elapsed_time_min = elapsed_time_sec / 60

# --- Final Summary ---
print("\n" + "=" * 40)
print("✅ ANALYSIS COMPLETE")
print(f"Total time taken: {elapsed_time_sec:.2f} seconds ({elapsed_time_min:.2f} minutes)")
print(f"Days processed:   {total_days_to_process}")
print(f"Total Rows Ingested: {total_rows:,}")
print(f"Final Report Shape: {session_report_df.shape} (Rows: {session_report_df.shape[0]}, Columns: {session_report_df.shape[1]})")
print("=" * 40)

--- Starting 75-Day Session Analysis ---
Processing range: 2025-09-01 to 2025-11-14
----------------------------------------


Processing Daily Sessions: 100%|██████████| 75/75 [03:12<00:00,  2.56s/day, Date: 2025-11-14]


✅ ANALYSIS COMPLETE
Total time taken: 192.18 seconds (3.20 minutes)
Days processed:   75
Total Rows Ingested: 51,788,694
Final Report Shape: (11770, 22) (Rows: 11770, Columns: 22)


In [ ]:
# session_report_df.to_csv('bcs_analysis_sessions.csv')
session_report_df.head(5)

,id,reg_num,customer,model,mode,session,start_time,end_time,duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
0,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,1,2025-09-01 05:30:00,2025-09-01 05:37:36,7.59,3.28,0.00,96.72,0.00,0.0,1.64,0.00,96.72,1.64,41.67,56.67,0.00,1.67
1,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,2,2025-09-01 05:39:01,2025-09-01 06:13:04,34.05,0.05,0.00,38.84,61.11,0.0,0.05,0.05,99.90,0.00,7.10,92.85,0.05,0.00
2,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,3,2025-09-01 06:13:05,2025-09-01 06:13:06,0.02,0.00,0.00,0.00,100.00,0.0,0.00,0.00,100.00,0.00,0.00,100.00,0.00,0.00
3,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,4,2025-09-01 06:13:07,2025-09-01 07:22:50,69.71,0.03,16.74,32.25,50.99,0.0,0.03,0.00,99.97,0.00,0.00,49.75,21.03,29.22
4,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,5,2025-09-01 07:23:05,2025-09-01 18:03:03,639.97,34.98,53.80,11.22,0.00,0.0,0.04,84.33,15.61,0.01,75.65,18.80,3.82,1.72


In [21]:
import pandas as pd

def weighted_avg_factory(df, weight_col='duration'):
    """
    Creates a callable function for pandas aggregation that calculates 
    the weighted average of a series using a specific weight column from the 
    original DataFrame (df) based on the group's indices.
    """
    def weighted_avg(series):
        # Retrieve the original 'duration' (weights) for the rows in the current group (series)
        weights = df.loc[series.index, weight_col]
        
        # Guard against division by zero (e.g., if total duration is 0)
        if weights.sum() == 0:
            return 0.0
            
        # Weighted average calculation
        return ((series * weights).sum() / weights.sum()).round(4)

    return weighted_avg

# --- ASSUMING session_report_df is your input DataFrame (the 75-day report) ---
# NOTE: Replace 'session_report_df' with the actual variable name from your script (e.g., the output of run_multi_day_session_analysis)
df = session_report_df 

# Ensure 'duration' is numeric and set up the weighted average function
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
w_avg = weighted_avg_factory(df, 'duration')

# Define all percentage columns for aggregation
pct_cols = [col for col in df.columns if col.endswith('_pct')]

# Define the full aggregation dictionary
agg_dict = {
    'reg_num': 'first',
    'customer': 'first',
    'model': 'first',
    'session': 'count',  # Becomes total_sessions
    'duration': 'sum'   # Becomes total_duration
}

# Add weighted average for all percentage columns
weighted_agg_dict = {col: w_avg for col in pct_cols}
agg_dict.update(weighted_agg_dict)

# 1. Perform aggregation grouped by 'id' and 'mode'
report_agg = df.groupby(['id', 'mode']).agg(agg_dict).reset_index()

# 2. Final Formatting and Column Renaming/Reordering
report_agg = report_agg.rename(columns={'session': 'total_sessions', 'duration': 'total_duration'})
report_agg['total_duration'] = report_agg['total_duration'].round(2)

report_agg['maxtemp_bucket_lt28_pct'] = report_agg['maxtemp_bucket_lt28_pct'].round(2)
report_agg['maxtemp_bucket_28_32_pct'] = report_agg['maxtemp_bucket_28_32_pct'].round(2)
report_agg['maxtemp_bucket_32_35_pct'] = report_agg['maxtemp_bucket_32_35_pct'].round(2)
report_agg['maxtemp_bucket_35_40_pct'] = report_agg['maxtemp_bucket_35_40_pct'].round(2)
report_agg['maxtemp_bucket_gt40_pct'] = report_agg['maxtemp_bucket_gt40_pct'].round(2)

report_agg['temp_delta_bucket_lt2_pct'] = report_agg['temp_delta_bucket_lt2_pct'].round(2)
report_agg['temp_delta_bucket_2_5_pct'] = report_agg['temp_delta_bucket_2_5_pct'].round(2)
report_agg['temp_delta_bucket_5_8_pct'] = report_agg['temp_delta_bucket_5_8_pct'].round(2)
report_agg['temp_delta_bucket_gt8_pct'] = report_agg['temp_delta_bucket_gt8_pct'].round(2)

report_agg['volt_delta_bucket_0_10_pct'] = report_agg['volt_delta_bucket_0_10_pct'].round(2)
report_agg['volt_delta_bucket_10_20_pct'] = report_agg['volt_delta_bucket_10_20_pct'].round(2)
report_agg['volt_delta_bucket_20_30_pct'] = report_agg['volt_delta_bucket_20_30_pct'].round(2)
report_agg['volt_delta_bucket_gt30_pct'] = report_agg['volt_delta_bucket_gt30_pct'].round(2)


# Reorder columns as requested
final_columns = [
    'id', 'reg_num', 'customer', 'model', 'mode', 'total_sessions', 'total_duration'
] + pct_cols

report_agg = report_agg[final_columns]

In [ ]:
# report_agg.to_csv('bcs_analysis_report_v1.csv')
report_agg.head()

,id,reg_num,customer,model,mode,total_sessions,total_duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
0,10,REGNUM_10,CUST_10,MDL_10,CHARGING,20,681.50,18.18,72.23,9.59,0.00,0.00,41.03,53.94,4.99,0.04,70.45,17.11,4.32,8.11
1,10,REGNUM_10,CUST_10,MDL_10,DISCHARGING,33,3981.14,52.91,42.04,5.05,0.00,0.00,48.95,46.84,4.04,0.17,80.58,16.64,0.46,0.35
2,11,AP39WF8593,FB Guntur,12.5,CHARGING,329,18984.66,1.79,20.42,49.12,27.23,1.44,3.17,75.90,20.84,0.09,47.02,38.35,3.25,11.38
3,11,AP39WF8593,FB Guntur,12.5,DISCHARGING,397,80097.73,5.89,47.43,39.58,7.03,0.07,17.12,77.47,5.41,0.00,81.63,13.81,1.93,2.63
4,12,TN14AR3907,FB Bangalore,12.5,CHARGING,270,11245.60,3.48,28.12,44.66,21.45,2.30,3.03,54.65,40.19,2.13,25.13,46.62,10.19,18.06


In [26]:
session_report_df[(session_report_df.maxtemp_bucket_35_40_pct>40)|(session_report_df.maxtemp_bucket_gt40_pct>40)].to_csv('bcs_session_maxT.csv')
session_report_df[(session_report_df.maxtemp_bucket_35_40_pct>40)|(session_report_df.maxtemp_bucket_gt40_pct>40)].head()

,id,reg_num,customer,model,mode,session,start_time,end_time,duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
1,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,2,2025-09-01 05:39:01,2025-09-01 06:13:04,34.05,0.05,0.00,38.84,61.11,0.0,0.05,0.05,99.90,0.0,7.10,92.85,0.05,0.00
2,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,3,2025-09-01 06:13:05,2025-09-01 06:13:06,0.02,0.00,0.00,0.00,100.00,0.0,0.00,0.00,100.00,0.0,0.00,100.00,0.00,0.00
3,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,4,2025-09-01 06:13:07,2025-09-01 07:22:50,69.71,0.03,16.74,32.25,50.99,0.0,0.03,0.00,99.97,0.0,0.00,49.75,21.03,29.22
28,3,HR55AY7626,ZB Gurgaon,12.5,CHARGING,2,2025-09-01 06:22:48,2025-09-01 07:00:22,37.56,0.05,0.00,11.82,88.14,0.0,0.05,0.14,99.81,0.0,55.91,44.09,0.00,0.00
29,3,HR55AY7626,ZB Gurgaon,12.5,DISCHARGING,3,2025-09-01 07:00:23,2025-09-01 07:01:41,1.29,1.52,0.00,0.00,98.48,0.0,1.52,0.00,98.48,0.0,100.00,0.00,0.00,0.00


In [27]:
session_report_df[(session_report_df.temp_delta_bucket_5_8_pct>40)|(session_report_df.temp_delta_bucket_gt8_pct>40)].to_csv('bcs_session_deltaT.csv')
session_report_df[(session_report_df.temp_delta_bucket_5_8_pct>40)|(session_report_df.temp_delta_bucket_gt8_pct>40)].head()

,id,reg_num,customer,model,mode,session,start_time,end_time,duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
0,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,1,2025-09-01 05:30:00,2025-09-01 05:37:36,7.59,3.28,0.00,96.72,0.00,0.0,1.64,0.00,96.72,1.64,41.67,56.67,0.00,1.67
1,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,2,2025-09-01 05:39:01,2025-09-01 06:13:04,34.05,0.05,0.00,38.84,61.11,0.0,0.05,0.05,99.90,0.00,7.10,92.85,0.05,0.00
2,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,3,2025-09-01 06:13:05,2025-09-01 06:13:06,0.02,0.00,0.00,0.00,100.00,0.0,0.00,0.00,100.00,0.00,0.00,100.00,0.00,0.00
3,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,4,2025-09-01 06:13:07,2025-09-01 07:22:50,69.71,0.03,16.74,32.25,50.99,0.0,0.03,0.00,99.97,0.00,0.00,49.75,21.03,29.22
5,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,6,2025-09-01 18:03:04,2025-09-01 18:05:26,2.36,0.74,0.74,98.53,0.00,0.0,0.74,0.00,98.53,0.74,35.56,62.96,0.74,0.74


In [28]:
session_report_df[(session_report_df.volt_delta_bucket_20_30_pct>40)|(session_report_df.volt_delta_bucket_gt30_pct>40)].to_csv('bcs_session_deltaV.csv')
session_report_df[(session_report_df.volt_delta_bucket_20_30_pct>40)|(session_report_df.volt_delta_bucket_gt30_pct>40)].head()

,id,reg_num,customer,model,mode,session,start_time,end_time,duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
17,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,18,2025-09-01 19:32:06,2025-09-01 20:03:09,31.05,0.00,12.10,87.90,0.0,0.0,0.00,0.00,99.94,0.06,0.00,45.21,51.71,3.08
19,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,20,2025-09-01 20:04:05,2025-09-01 20:32:22,28.29,0.07,72.54,27.40,0.0,0.0,0.07,32.33,67.61,0.00,1.21,2.28,8.85,87.66
51,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,20,2025-09-02 19:00:55,2025-09-02 19:34:57,34.03,0.00,67.06,32.94,0.0,0.0,0.00,78.84,21.11,0.05,4.71,2.60,21.45,71.24
65,3,HR55AY7626,ZB Gurgaon,12.5,CHARGING,10,2025-09-02 20:07:28,2025-09-02 20:53:13,45.76,0.05,62.67,37.28,0.0,0.0,0.05,30.99,68.91,0.05,7.63,3.84,25.24,63.29
67,3,HR55AY7626,ZB Gurgaon,12.5,CHARGING,12,2025-09-02 20:53:16,2025-09-02 20:56:25,3.14,0.00,100.00,0.00,0.0,0.0,0.00,0.00,100.00,0.00,0.00,0.00,0.00,100.00
